In [1]:
from data.process import *

2024-07-20 10:20:24.041569: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-20 10:20:24.163171: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-20 10:20:24.230772: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-20 10:20:24.231251: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-20 10:20:24.310865: I tensorflow/core/platform/cpu_feature_gua

In [2]:
train_df, dev_df, test_df = retrieve_train_dev_test_as_category_for_xgboost()
X_train, Y_train = split_input_output(train_df)
X_dev, Y_dev = split_input_output(dev_df)

In [3]:
import xgboost as xgb
from model.xgb_opt import *

In [4]:
xgboost_params = {
    **xgb_fixed_params,
    # categorical features
    'enable_categorical': True,
    # fixed after secone optimization
    'colsample_bytree': 0.21039136022674532,
    'max_depth': 27,
    'min_child_weight': 36,
    # override fixed params
    'n_estimators': 5000,
    'early_stopping_rounds': 200,
}

In [5]:

scale_pos_weight = np.sum(Y_train == 0) / np.sum(Y_train == 1)

In [6]:
model = xgb.XGBClassifier(
    **xgboost_params,
    scale_pos_weight=scale_pos_weight,
)

boosting = model.fit(X_train, Y_train, eval_set=[(X_dev, Y_dev)], verbose=50)

[0]	validation_0-auc:0.68438
[50]	validation_0-auc:0.87765
[100]	validation_0-auc:0.88349
[150]	validation_0-auc:0.88704
[200]	validation_0-auc:0.88874
[250]	validation_0-auc:0.88965
[300]	validation_0-auc:0.89012
[350]	validation_0-auc:0.89039
[400]	validation_0-auc:0.89077
[450]	validation_0-auc:0.89122
[500]	validation_0-auc:0.89144
[550]	validation_0-auc:0.89170
[600]	validation_0-auc:0.89185
[650]	validation_0-auc:0.89194
[700]	validation_0-auc:0.89205
[750]	validation_0-auc:0.89211
[800]	validation_0-auc:0.89216
[850]	validation_0-auc:0.89222
[900]	validation_0-auc:0.89228
[950]	validation_0-auc:0.89234
[1000]	validation_0-auc:0.89237
[1050]	validation_0-auc:0.89240
[1100]	validation_0-auc:0.89236
[1150]	validation_0-auc:0.89236
[1200]	validation_0-auc:0.89240
[1250]	validation_0-auc:0.89238
[1300]	validation_0-auc:0.89233
[1350]	validation_0-auc:0.89230
[1400]	validation_0-auc:0.89230
[1450]	validation_0-auc:0.89228
[1461]	validation_0-auc:0.89228


In [7]:
# model.get_booster().save_model("xgboost_model_new.json")